In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf 
import cv2
import matplotlib
import matplotlib.pyplot as plt
import time
import random
import tensorflow_addons as tfa
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


AUTO = tf.data.AUTOTUNE

 semantic segmentation 
- architecture : U-net
    
  

# 🌈 Visualization 

In [ ]:
df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv') 

list = os.listdir("../input/sartorius-cell-instance-segmentation/train")

#transforme List ( af6ae867fe6e.png) into a list of id in the pandas dataframe (af6ae867fe6e) 
id_list = [l.replace('.png','') for l in list] 
print(type(id_list) , " 111")



def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)



# all image in test data have the same shape
shape = (520, 704)     
row = 0    


id_ = df.loc[row, 'id']  


print(type(id_))

UI = df["id"].unique() # ALL ID
print(len(UI))
    
    
img_masks = df.loc[df['id']==id_, 'annotation'].to_list()

print(len(img_masks))
all_masks = np.zeros(shape)


px = 1/plt.rcParams['figure.dpi']
rows = 3
fig, axarr = plt.subplots(rows, 2, figsize=(520*px, 704*px))
i=0
img = cv2.imread("../input/sartorius-cell-instance-segmentation/train/0030fd0e6378.png")
for mask in img_masks:
    all_masks += rle_decode(mask, shape)
    if i <=2:
        axarr[i,1].imshow(all_masks)
        axarr[i,1].axis('off')
        axarr[i,1].set_title(f'Masks {i}')
    
        axarr[i,0].imshow(img)
        axarr[i,0].axis('off')
    
        i+=1

print(all_masks.shape)
print(type(all_masks))

plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.show()

# 🌈  Other Visualization

In [ ]:
# test : 

N_exemple = 2
fig, axarr = plt.subplots(N_exemple, 2, figsize=(15, 40))
UI = df["id"].unique()        # type numpy.ndarray

    

for i in range(N_exemple):
    img_masks = df.loc[df['id']==UI[i], 'annotation'].to_list()
    print(len(img_masks))
    img = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{UI[i]}.png")
    all_masks = np.zeros(shape)
    
    for mask in img_masks:
        all_masks += rle_decode(mask, shape)
    
    
    #mask
    axarr[i, 1].imshow(all_masks)
    axarr[i, 1].axis('off')
    axarr[i, 1].set_title(f'Masks {i}')
    
    #image originale
    axarr[i, 0].imshow(img)
    axarr[i, 0].axis('off')
    axarr[i, 0].set_title(f'Masks {i}')
    

    
    
    
plt.tight_layout(h_pad=0.1, w_pad=0.1)
plt.show()



# 📁 Create TFrecord

In [ ]:
def _bytes_feature(value):         # S/O la doc tensorflow et le livre pour convertyre en byte pour le TFRECORD
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
  
    
        
def build_tfrecord_2():
    df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')    #datafrme
    start = time.time()
    shape = (520, 704)     # all image in test data have the same shape
    UI = df["id"].unique() # Nombre de De photo / id differente        #606 
    
    df1 = df.groupby('id',as_index=False,sort=False).last()
    lbl = LabelEncoder()
    df1["cell_type"] = lbl.fit_transform(df1["cell_type"])
    
    option = tf.io.TFRecordOptions(compression_level=2, compression_type="ZLIB")
    
    with tf.io.TFRecordWriter("Train2.tfrec", options=option) as writer:
        for i in tqdm(range(len(UI))):
            img_masks = df.loc[df['id']==UI[i], 'annotation'].to_list() # all mask in 1 list 
            img = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{UI[i]}.png")      # Image 
            all_masks = np.zeros(shape, dtype=np.float32)
            img = np.true_divide(img, 255, dtype=np.float32)
        
            
            for mask in img_masks:
                all_masks += rle_decode(mask, shape) # get the mask
            
            all_masks[all_masks > 1] = 1   #because some mask intercepts
            
            data = {'image': _bytes_feature(img.tobytes()),
                    'mask': _bytes_feature(all_masks.tobytes()),
                    'label': _int64_feature(df1["cell_type"].iloc[i]),
                    }
        
            Data = tf.train.Example(features=tf.train.Features(feature=data))
            Data = Data.SerializeToString()

            writer.write(Data)

    elapsed = time.time()
    elapsed = elapsed - start
    print("Time spent: ", elapsed)
        
        

# 📖 Read DS and data augmentation

In [ ]:


#DS for mask

def decode_mask(image_data):
    image = tf.io.decode_raw(image_data['image'], tf.float32)
    image = tf.reshape(image, [520,704,3])
    mask = tf.io.decode_raw(image_data['mask'], tf.float32)
    mask = tf.reshape(mask, [520,704,1])
    return (image, mask)


def read_mask(exemple):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'mask': tf.io.FixedLenFeature([], tf.string)
    }
    Exemple = tf.io.parse_single_example(exemple , image_feature_description)
    return decode_mask(Exemple)



# DS for label


def read_label(exemple):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'mask': tf.io.FixedLenFeature([], tf.string)
    }
    Exemple = tf.io.parse_single_example(exemple , image_feature_description)
    #image, label= decode_label(Exemple) 
    return decode_label(Exemple)


def decode_label(image_data):
    image = tf.io.decode_raw(image_data['image'], tf.float32)
    image = tf.reshape(image, [520,704,3])
    label = image_data['label']
    return (image, label)




def augment3(image, mask):
    # Make a new seed.
    new_seed= tf.random.uniform([2], minval=0, maxval=200000, dtype=tf.dtypes.int32)
    #new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]
    
    #random rotation 
    N=tf.random.uniform([1], minval=-3, maxval=3, dtype=tf.dtypes.float32)
    image = tfa.image.rotate( image,N[0])
    mask = tfa.image.rotate( mask,N[0])
   
    #random contrast ?
    
    
    # random HUE ? 
    
    
    
    # Random brightness ( uniquement pour image )
    image = tf.image.stateless_random_brightness(image, max_delta=0.3, seed=new_seed)
    #label = tf.image.stateless_random_brightness(label, max_delta=0.5, seed=new_seed)
    
    # flip up down
    image = tf.image.stateless_random_flip_up_down(image, new_seed)
    mask =  tf.image.stateless_random_flip_up_down(mask, new_seed)
    
    # flip left right
    image = tf.image.stateless_random_flip_left_right(image, new_seed)
    mask = tf.image.stateless_random_flip_left_right(mask, new_seed)
    
    image = tf.clip_by_value(image, 0, 255)
    #label = tf.clip_by_value(label, 0, 255)
    
    return image, mask


def augment3L(image ,label):
    # Make a new seed.
    new_seed= tf.random.uniform([2], minval=0, maxval=200000, dtype=tf.dtypes.int32)
    #new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]
    
    #random rotation 
    N=tf.random.uniform([1], minval=-3, maxval=3, dtype=tf.dtypes.float32)
    image = tfa.image.rotate( image,N[0])
   
    #random contrast ?
    
    
    # random HUE ? 
    
    
    
    # Random brightness ( uniquement pour image )
    image = tf.image.stateless_random_brightness(image, max_delta=0.4, seed=new_seed)
    #label = tf.image.stateless_random_brightness(label, max_delta=0.5, seed=new_seed)
    
    # flip up down
    image = tf.image.stateless_random_flip_up_down(image, new_seed)

    
    # flip left right
    image = tf.image.stateless_random_flip_left_right(image, new_seed)

    
    image = tf.clip_by_value(image, 0, 255)
    #label = tf.clip_by_value(label, 0, 255)
    
    return image, label



def load_dataset(path , Augment = False , Big = False , label=True):
    # Label = TRUE  :  image and label
    # Label = False : image and Mask
    
    
    dataset = tf.data.TFRecordDataset(path,compression_type="ZLIB", num_parallel_reads=AUTO)
    if label:
        dataset = dataset.map(read_label, num_parallel_calls= AUTO)
    else:
        dataset = dataset.map(read_mask, num_parallel_calls= AUTO)
    if Big:
        dataset = dataset.repeat(6)
    if Augment :
        if label:
            dataset = dataset.map(augment3L,num_parallel_calls = AUTO) 
        else:
            dataset = dataset.map(augment3,num_parallel_calls = AUTO)
    return dataset


def view_dataset(path, Augment = False ,big = False, label=True ):
    data = load_dataset(path, Augment = Augment  , Big = big, label=label)  # class 'tensorflow.python.data.ops.dataset_ops.ParallelMapDataset'   

    N = 5
    i = 0
    

    ds = data.take(N)
    if label:
        fig, axarr = plt.subplots(N,1, figsize=(15, 40))
        for image ,label  in ds:
            print(image.shape , type(image))
            print(label.shape)
        
            #image 
            axarr[i].imshow(image)
            axarr[i].axis('off')
            axarr[i].set_title(f'Masks {label}')
            i+=1
    else:
        fig, axarr = plt.subplots(N,2, figsize=(15, 40))
        for image ,mask  in ds:
            print(image.shape , type(image))
            print(mask.shape)
        
            reshape = tf.reshape(mask ,[520*704])
            Unique = tf.unique(reshape)
            print(mask.shape)
            print(Unique)
        
            #mask
            axarr[i, 1].imshow(mask)
            axarr[i, 1].axis('off')
            axarr[i, 1].set_title(f'Masks {i}')
    
            #image 
            axarr[i, 0].imshow(image)
            axarr[i, 0].axis('off')
            axarr[i, 0].set_title(f'image {i}')
            i+=1
    
    plt.tight_layout(h_pad=0.1, w_pad=0.1)
    plt.show()
    
    return(data)
    # pour afficher 1 seule image
''' fig, axs = plt.subplots(1, 2,figsize=(20, 20))                
    axs[0].imshow(image)
    axs[0].axis('off')
    axs[1].imshow(np.sum(mask, axis=-1))
    axs[1].axis('off')'
    plt.show()'''


DCNN for classification


In [ ]:
def DCNN(DS):

    
    inputs = tf.keras.Input(shape=[520, 704, 3])
    x = inputs
    x = tf.keras.layers.Conv2D(20, kernel_size =(3, 3),strides=(1, 1) , padding="same" )(x)
    x = tf.keras.layers.MaxPooling2D( pool_size=(2, 2), strides=(2, 2))(x)
    
    x = tf.keras.layers.Conv2D(40, kernel_size =(3, 3),strides=(1, 1) , padding="same")(x)
    x = tf.keras.layers.MaxPooling2D( pool_size=(2, 2), strides=(2, 2))(x)
    
    x = tf.keras.layers.Conv2D(60, kernel_size =(3, 3),strides=(1, 1) , padding="same" )(x)
    x = tf.keras.layers.MaxPooling2D( pool_size=(2, 2), strides=(2, 2))(x)
    
    x = tf.keras.layers.Conv2D(80, kernel_size =(3, 3),strides=(1, 1) , padding="same" )(x)
    x = tf.keras.layers.MaxPooling2D( pool_size=(2, 2), strides=(2, 2))(x)
    
    
    x = tf.keras.layers.Flatten()(x)
    
    x = tf.keras.layers.Dense(10, activation='elu', kernel_initializer=tf.keras.initializers.LecunNormal())(x)   # tf.nn.leaky_relu
    x = tf.keras.layers.Dropout(rate=0.4)(x)
    
    x = tf.keras.layers.Dense(512, activation='elu', kernel_initializer=tf.keras.initializers.LecunNormal())(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)
    
    x = tf.keras.layers.Dense(512, activation='elu', kernel_initializer=tf.keras.initializers.LecunNormal())(x)
    x = tf.keras.layers.Dropout(rate=0.4)(x)
    
                               
    outputs = tf.keras.layers.Dense(3 , activation = 'softmax')(x) 

                                     
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    

    model.summary()



    # model compile and fit
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)


    start = time.time()
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer=optimizer, metrics=["accuracy"])

    history = model.fit(DS ,epochs=10 )

    model.save('./Test_1.h5')
    


    # plot ressult

    elapsed = time.time()
    elapsed = elapsed - start
    print("Time spent: ", elapsed)

    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)  # set the vertical range to [0-1]
    plt.show()
    


In [ ]:
if __name__ == "__main__":
    path = "./Train2.tfrec"
    
    build_tfrecord_2()
    #ds = view_dataset(path,Augment = True, big = True ,label=False)
    
    #DCNN(ds.batch(100).prefetch(AUTO))
